In [1]:
from tqdm import tqdm
import sys
sys.path.append('../30_data_tools')
from helper import load_dotenv
from pathlib import Path
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim
import pandas as pd
import cv2

In [2]:
dotenv = load_dotenv()

In [3]:
halftone_images = list(dotenv['DATA_DIR'].glob('./*/ps2400dpi150lpi/*.4c_300.jpg'))

In [4]:
values = []

for ps_img_path in tqdm(halftone_images[:5]):
    vps_img_path = ps_img_path.parent.parent / 'vps2400dpi150lpi' / ps_img_path.name.replace('.4c.4c_300.jpg','.4c_300.jpg')

    ps_img = np.array(Image.open(ps_img_path))
    vps_img = np.array(Image.open(vps_img_path))
    
    if vps_img.shape[0] > ps_img.shape[0]:
        diff = round((vps_img.shape[0] - ps_img.shape[0]) * 0.5)
        
        vps_img = vps_img[
            diff:diff+ps_img.shape[0],
            :,
            :
        ]
    
    if vps_img.shape[1] > ps_img.shape[1]:
        diff = round((vps_img.shape[1] - ps_img.shape[1]) * 0.5)
        
        vps_img = vps_img[
            :,
            diff:diff+ps_img.shape[1],
            :
        ]

    for i in range(ps_img.shape[2]):
        ssim_value = ssim(
            ps_img[:,:,i],
            vps_img[:,:,i]
        )

        small_ssim_value = ssim(
           cv2.resize(ps_img[:,:,i], (0,0), fx=0.5, fy=0.5),
           cv2.resize(vps_img[:,:,i], (0,0), fx=0.5, fy=0.5) 
        )
    
        values.append((
            ps_img_path.name,
            ['C','M','Y','K'][i],
            ssim_value,
            small_ssim_value
        ))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:16<00:00,  3.38s/it]


In [5]:
df = pd.DataFrame(
    values,
    columns=['img','separation','ssim_300','ssim_150']
)

In [6]:
df.describe()

,ssim_300,ssim_150
count,20.000000,20.000000
mean,0.562961,0.871760
std,0.169140,0.077037
min,0.230082,0.715806
25%,0.419084,0.819366
50%,0.570990,0.893635
75%,0.648606,0.937861
max,0.843849,0.966095


In [7]:
df.loc[
    :,
    ['separation','ssim_300','ssim_150']
].groupby('separation').mean()

,ssim_300,ssim_150
separation,,
C,0.757056,0.946942
K,0.397994,0.778893
M,0.538122,0.914227
Y,0.558672,0.846978
